In [2]:
import csv


In [3]:
with open('5-Hopkins_clinical_panel_submission_template_testfile.csv', 'r') as my_csv_file:
    dic_patients_list = [item for item in csv.DictReader(my_csv_file)]


In [4]:
true_dic_patients_list = [dict(item) for item in dic_patients_list]

In [5]:
#print(true_dic_patients_list)

In [6]:
#The rows are the patients (1-106, number based on CAGI-4). 

#A list of errors which we will display later if the file fails validation. 
error_list=[]

#Test to see if there are empty values in the table.
for item in true_dic_patients_list:
    for patient_file in item.keys():
        if item[patient_file] == '':
            error_list.append('Empty value at row, column: '+ 
                              item['patient']+" , "+ patient_file + "; Please leave '*' for empty value")

#Test probability values to make sure they are legitimate.
for item in true_dic_patients_list:
    for patient_file in item.keys():
        if (patient_file[-1] =='P' and item[patient_file] != '*')and (float(item[patient_file]) > 1 or float(item[patient_file])<0):
            error_list.append('Probability value error for row/patient: '+ item['patient'] +
                  ', cell coordinates: '+ patient_file)      

#Test to see if SD values are legtimate:
#Merges and groups data in columns by disorder category.
patient_num_val = []
test_dict ={}
syndrome_dict ={'0':'1', '3':'2', '6':'3', '9': '4', 
                '12':'5', '15':'6', '18':'7', 
                '21':'8', '24':'9','27': '10',
               '30': '11', '33':'12', '36': '13','39':'14'}
patient_list = [patient_row['patient'] for patient_row in true_dic_patients_list]
print(patient_list)

for patient_row in true_dic_patients_list:
    patient_val_list =[]
    for patient_val in patient_row.keys():
        if (patient_val[0] != 'p' and patient_val !='C'):
            test_dict[patient_val] = patient_row[patient_val]
            patient_val_list.append(patient_row[patient_val])
    patient_num_val.append(patient_val_list)

    print(patient_num_val, '\n')
print(test_dict)

for patient in patient_num_val:
    for position in range(len(patient)-3):
        if position % 3 == 0 and patient[position+1] =="*" and patient[position]!='*':
            error_list.append("Error: You can't predict a SD without a corresponding prediction in the previous column; " + 'Patient ID: '+ 
                              patient_list[patient_num_val.index(patient)]+ 
                              ' Disorder:'+ syndrome_dict[str(position)])
        elif position % 3 == 0 and patient[position] =='*' and patient[position+1] !="*":
            error_list.append('Error: SD without P, '+ 
                              'Patient ID: '+ 
                              patient_list[patient_num_val.index(patient)]+ 
                              ' Disorder:'+ 
                              syndrome_dict[str(position)])
        elif position % 3 == 0 and patient[position] !='*' and int(patient[position+1]) < 0:
            error_list.append('Error: SD must be >=0; '+ 
                              'Patient ID: '+ 
                              patient_list[patient_num_val.index(patient)]+ 
                              ' Disorder:'+ 
                              syndrome_dict[str(position)])
         


#Variant Testing
#Merges and groups data in columns by disorder category.




#Print errors
if len (error_list) >0:
    print('Validation failed with errors:', '\n')
    for item in error_list:
        print(item)
else:
    print("Validation passed with no errors.")
    

['P1', 'P2']
[['0.5', '2', 'charararar', '5', '2', 'google', '0.2', '2', 'goolish', '0.3', '-2', 'dynamite', '0.2', '*', 'Nobel', '*', '2', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '2', '*', '*']] 

[['0.5', '2', 'charararar', '5', '2', 'google', '0.2', '2', 'goolish', '0.3', '-2', 'dynamite', '0.2', '*', 'Nobel', '*', '2', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '2', '*', '*'], ['*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']] 

{' 1-P': '*', '1-SD': '*', '1-V': '*', ' 2-P': '*', '2-SD': '*', '2-V': '*', ' 3-P': '*', '3-SD': '*', '3-V': '*', '4-P': '*', '4-SD': '*', '4-V': '*', '5-P': '*', '5-SD': '*', '5-V': '*', '6-P': '*', '6-SD': '*', '6-V': '*', '7-P': '*